In [13]:
import os
from dotenv import load_dotenv
from yandex_chain import YandexLLM, YandexEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.pdf import UnstructuredPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

load_dotenv()

True

In [9]:
embeddings = YandexEmbeddings(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))
# text_splitter = CharacterTextSplitter(chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)


for path in os.listdir('../files/txts'):
    txt_path = '../files/txts/'+path
    loader = TextLoader(txt_path)
    document = loader.load()
    doc = text_splitter.split_documents(document)
    db = FAISS.from_documents(doc, embeddings)
    print(db.index.ntotal)

73
52
68
68
161
127
174
197
218
182
324


In [10]:
db.save_local('../files', index_name='finance_docs')

In [12]:
query = "Что по гособлигациям?"
print(f"+ Запрос: {query}")

print(" + Получение релевантных документов")
retriever = db.as_retriever()
result = retriever.get_relevant_documents(query)
for x in result:
    print(f"{x}\n------\n")

+ Запрос: Что по гособлигациям?
 + Получение релевантных документов
page_content='договорные  условия  финансового  актива  обуславливают  получение  в  указанные  даты \nденежных  потоков,  являющихся  исключительно  платежами в  счет  основной суммы долга и \nпроцентов на непогашенную часть основной суммы долга. \n\uf0b7 \n \nПри  первоначальном  признании  Группа  может  принять  решение,  без  права  его  последующей \nотмены, представлять в  составе  прочего  совокупного дохода  изменения  справедливой стоимости \nинвестиции в долевой инструмент, не предназначенный для торговли. Такое решение принимается \nпо каждому инструменту в отдельности. \n \n \n22' metadata={'source': '../files/txts/Консолидированная финансовая отчетность за год закончившийся 31 декабря 2022 г. (в рублях)_.txt'}
------

page_content='учитываются  в  российских  депозитариях,  произошла  автоматически.  Конвертация  депозитарных \nрасписок,  права  на  которые  учитываются  в  иностранных  депозитариях,  был

/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
template = """Ответь на вопрос, основываясь только на текст ниже:
{context}

Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = YandexLLM(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

print(" + Запуск LLM Chain")
res = chain.invoke(query)
print(f" + Ответ: {res}")

 + Запуск LLM Chain
 + Ответ: Исходя из предоставленного текста, можно сделать вывод, что по состоянию на 31 декабря 2022 года **по гособлигациям не проводились торги**, но они были возобновлены в конце марта того же года. При этом были **действительны ограничения на сделки с ценными бумагами,** которые совершались нерезидентами, так как торги проходили на Московской бирже. 

Вероятно, вы имеете в виду что-то ещё — пожалуйста, уточните ваш вопрос.
